# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
pip install plotly

     --------------------------------------- 15.5/15.5 MB 50.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install hvplot

     ---------------------------------------- 3.2/3.2 MB 25.4 MB/s eta 0:00:00
  Using cached param-1.13.0-py2.py3-none-any.whl (87 kB)
  Using cached holoviews-1.16.2-py2.py3-none-any.whl (4.3 MB)
     ---------------------------------------- 7.8/7.8 MB 38.5 MB/s eta 0:00:00
  Using cached colorcet-3.0.1-py2.py3-none-any.whl (1.7 MB)
  Using cached panel-1.2.0-py2.py3-none-any.whl (20.0 MB)
  Using cached xyzservices-2023.5.0-py3-none-any.whl (56 kB)
  Using cached contourpy-1.1.0-cp310-cp310-win_amd64.whl (470 kB)
  Using cached pyct-0.5.0-py2.py3-none-any.whl (15 kB)
  Using cached pyviz_comms-2.3.2-py2.py3-none-any.whl (79 kB)
  Using cached typing_extensions-4.7.1-py3-none-any.whl (33 kB)
  Using cached Markdown-3.4.3-py3-none-any.whl (93 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached mdit_py_plugins-0.4.0-py3-none-any.whl (54 kB)
  Using cached linkify_it_py-2.0.2-py3-none-any.whl (19 kB)
  Using cached uc_micro_py-1.0.2-py3-none-any.whl (6.2 kB)


In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,293.40,28,28,7.13,RU,1688920320
1,1,port-aux-francais,-49.3500,70.2167,274.35,56,56,7.50,TF,1688920320
2,2,shiloh,38.5614,-89.8973,300.41,75,75,0.89,US,1688920280
3,3,teguise,29.0605,-13.5640,299.27,20,20,10.29,ES,1688920320
4,4,lae,-6.7221,146.9847,297.42,99,99,1.93,PG,1688920320


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [47]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504,
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityPlot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [48]:
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,udachny,66.4167,112.4000,293.40,28,28,7.13,RU,1688920320
1,1,port-aux-francais,-49.3500,70.2167,274.35,56,56,7.50,TF,1688920320
2,2,shiloh,38.5614,-89.8973,300.41,75,75,0.89,US,1688920280
3,3,teguise,29.0605,-13.5640,299.27,20,20,10.29,ES,1688920320
4,4,lae,-6.7221,146.9847,297.42,99,99,1.93,PG,1688920320
...,...,...,...,...,...,...,...,...,...,...
594,594,novyy turtkul',41.5500,61.0167,303.10,0,0,4.68,UZ,1688920490
595,595,magadan,59.5667,150.8000,285.67,100,100,1.74,RU,1688920490
596,596,nagtipunan,16.2167,121.6000,296.52,8,8,0.55,PH,1688920491
597,597,soufriere,13.8562,-61.0566,303.96,20,20,10.29,LC,1688920491


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [49]:
# Narrow down cities that fit criteria and drop any results with null values- 
#max temp 27 C ~ 80.6 F; min temp 21C ~ 69.8 F; wind < 4.5; cloudiness = 0

idealCityDF= city_data_df.loc[
    (city_data_df["Max Temp"] > 307.20) & (city_data_df["Max Temp"]< 288.77) \
    & (city_data_df [ "Wind Speed"]< 7.72) \
    & (city_data_df["Cloudiness"]==0)
]

# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

0


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [50]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accomodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row ["Lat"]
    longitude = row ["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = "circle:{longitude}, {latitude}, {radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City,Country,Lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
#%%capture --no-display

# Configure the map plot
hotelPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    scale= 1,
    color = "City",
    tiles = "OSM",
    frame_width = 896,
    frame_height = 504,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotelPlot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country)